## Search engine using Tools and Agents

- Install `wikipedia` and `yfinance` library
- Import tools `WikipediaQueryRun` and `YahooFinanceNewsTool`

#### Import tools and utilities

In [14]:
from langchain_community.tools import WikipediaQueryRun, YahooFinanceNewsTool
from langchain_community.utilities import WikipediaAPIWrapper, GoogleSearchAPIWrapper

#### Create Tools

In [19]:
wiki_api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_api_wrapper)

yahoo_fin_tool = YahooFinanceNewsTool()

tools = [wiki_tool, yahoo_fin_tool]

#### Create Custom RAG tool

In [20]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.tools.retriever import create_retriever_tool

docs = PyPDFDirectoryLoader("../files/health-reports").load()

sp = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=50)
docs_chunks = sp.split_documents(docs)

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vector_store = FAISS.from_documents(docs, embeddings)

retriever = vector_store.as_retriever()
tag_tool = create_retriever_tool(retriever, "health-report-tool", "Tool to reteive health information from PDF")

tools.append(tag_tool)

In [21]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\Practice\\gen-ai-langchain\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 YahooFinanceNewsTool(),
 Tool(name='health-report-tool', description='Tool to reteive health information from PDF', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001B17F5DF010>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001B171032FB0>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x000001B17F97E680>, retriever=VectorStoreRetriever(tags=['FAISS', 

#### Run all Tools with Agents and LLMs

In [22]:
from langchain_groq import ChatGroq

In [23]:
llm = ChatGroq(model="llama-3.1-70b-versatile")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001B172B956F0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001B16FCB85E0>, model_name='llama-3.1-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

Import agent Prompt template

In [24]:
from langchain import hub
# Import PromptTemplate from langchain 
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

Create Agent

In [25]:
from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

Create Agent executor       

In [26]:
from langchain.agents import AgentExecutor

executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [28]:
executor.invoke({"input": "How is Ranjan's health?"})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `health-report-tool` with `{'query': 'Ranjan'}`


Name                               : MR TRUPTI RANJAN PADHI
Registration No              : MH013948892
Patient Episode              : H01000137707
Referred By                    : HC MHW
Receiving Date               : 13 Sep 2024 08:24Age                    : 33 Yr(s) Sex :Male
Lab No              : 21240900934
Collection Date : 13 Sep 2024 07:29
Reporting Date : 13 Sep 2024 11:01
BLOOD BANK
 
 
    Blood Group & Rh Typing (Agglutination by gel/tube technique)  Specimen-Blood 
 
    Anti A                   Negative               Anti D                   Positive 
    Anti B                   Negative               Ctr                      Negative 
    Anti A1                  Negative               A1 Cell                  Positive 
    Anti H                   Positive               B Cell                   Positive 
                                                    O Cell                   Negative 
 
    Blood Group & 

{'input': "How is Ranjan's health?",
 'output': "Based on the provided health reports, here are the key findings:\n\n1. Blood Group and Rh Typing: The patient's blood group is O Rh(D) Positive.\n2. Lipid Profile: The patient's lipid profile shows elevated levels of total cholesterol (229 mg/dl), triglycerides (283 mg/dl), and LDL cholesterol (165 mg/dl). The HDL cholesterol level is 42 mg/dl. The total cholesterol to HDL cholesterol ratio is 5.5, and the LDL to HDL cholesterol ratio is 3.9. These values indicate a moderate to high risk of cardiovascular disease.\n3. Liver Function Test: The patient's liver function test shows elevated levels of SGOT/AST (52.0 U/L) and SGPT/ALT (136.0 U/L). These values indicate liver damage or inflammation.\n4. Erythrocyte Sedimentation Rate (ESR): The patient's ESR is 10 mm/1st hour, which is within the normal range.\n\nOverall, the patient's health reports indicate that they have elevated lipid levels and liver enzymes, which may indicate a risk of c

In [35]:
executor.invoke({"input": "Who is Modi?"})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `wikipedia` with `{'query': 'Narendra Modi'}`


Page: Narendra Modi
Summary: Narendra Damodardas Modi (Gujarati: [ˈnəɾendɾə dɑmodəɾˈdɑs ˈmodiː] ; born 17 September 1950) is an Indian politician serving as the current prime minister of India since 26 May 2014. Modi was the chief minister of GujaratNarendra Modi is an Indian politician who is currently serving as the Prime Minister of India. He was born on September 17, 1950, and has been in office since May 26, 2014. Prior to becoming Prime Minister, Modi served as the Chief Minister of Gujarat.

> Finished chain.


{'input': 'Who is Modi?',
 'output': 'Narendra Modi is an Indian politician who is currently serving as the Prime Minister of India. He was born on September 17, 1950, and has been in office since May 26, 2014. Prior to becoming Prime Minister, Modi served as the Chief Minister of Gujarat.'}

In [36]:
executor.invoke({"input": "How is AAPL Stock today?"})

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



Invoking: `yahoo_finance_news` with `{'query': 'AAPL'}`


Warren Buffett Sold Half of Apple (AAPL) Shares in Q3
Vltava Fund, an investment management company, recently released its third-quarter 2024 investor letter. A copy of the letter can be downloaded here. The portfolio had a surprisingly favorable July and a negative September. The last quarter saw significant movements in stocks overall. The first part of August saw the most market volatility. The major stock […]

Cramer Says 'Hot Money' Flowing From Nvidia, Apple Into China, Focus On Alibaba 'If You Must'
Jim Cramer is sounding the alarm, but not in the way you might expect. For those bullish on Nvidia Corp (NASDAQ:NVDA) and Apple Inc (NASDAQ:AAPL), Cramer's latest take might be hard to swallow. Hot Money Streaming Out Of Nvidia, Apple In a tweet, ...AAPL stock market news was found, however, the current stock price was not present.

> Finished chain.


{'input': 'How is AAPL Stock Market today?',
 'output': 'AAPL stock market news was found, however, the current stock price was not present.'}